In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction\\exp'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

[2025-05-10 22:29:57,860 : INFO : __init__ : Logger has been set up successfully!]


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error: {e}")
  

[2025-05-10 22:29:58,422 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-05-10 22:29:58,443 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-05-10 22:29:58,465 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-05-10 22:29:58,468 : INFO : common : created directory at: artifacts]
[2025-05-10 22:29:58,471 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-05-10 22:30:00,256 : INFO : 635984761 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 150917
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d4689fbc1c1d4bf7e42c19e0c178f43236c6abd4a13d3d4c882374e09c179e67"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1015:1FE8C5:9FACD:163EED:681F860F
Accept-Ranges: by